[View in Colaboratory](https://colab.research.google.com/github/battlerhythm/tensorflow/blob/master/5.1%20Saver.ipynb)

In [1]:
from google.colab import files
files.upload()

Saving data.csv to data.csv


{'data.csv': b'0, 0, 1, 0, 0\n1, 0, 0, 1, 0\n1, 1, 0, 0, 1\n0, 0, 1, 0, 0\n0, 0, 1, 0, 0\n0, 1, 0, 0, 1\n'}

In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [0]:
# 모델을 저장할 때 쓸 변수를 하나 만듭니다.
# 이 변수는 학습에 직접 사용되지는 않고, 학습 횟수를 카운트하는 변수 입니다.
# 이를 위해 변수 정의시 trainable=False라는 옵션을 주었습니다.
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3]))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

# global_step매개변수에 앞서 정의한 global_step변수를 넘겨주었습니다.
# 이렇게 하면 최적화 함수가 학습용 변수들을 최적화할 때마다 global_step 변수의 값을 1씩 증가시키게 됩니다.
train_op = optimizer.minimize(cost, global_step=global_step)

In [14]:
sess = tf.Session()

# tf.global_variables는 앞서 정의한 변수들을 가져오는 함수입니다.
# 이 함수를 써서 앞서 정의한 변수들을 모두 가져와서, 이후 변수들을 파일에 저장하거나 
# 이전에 학습한 결과를 불러와 담는 변수들로 사용합니다.
saver = tf.train.Saver(tf.global_variables())

# 다음코드는 ./model 디렉토리에 기존에 학습해둔 모델이 있는지를 확인해서 모델이 있다면
# saver.restore 함수를 이용헤 학습된 값들을 불러오고, 아니면 변수를 새로 초기화합니다.
# 학습된 모델을 저장한 파일을 체크포인트파일(Checkpoint file) 이라고 합니다.
ckpt = tf.train.get_checkpoint_state('./model')

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 계속 보아온 바와 같은 방법으로 간단하게 최적화를 수행합니다.
# 이전과는 달리 step값이 아니라 global_step값을 이용해 학습을 몇 번째 진행하고 있는지를 출력해 줍니다.
# global_step은 텐서 타입의 변수이므로 값을 가져올 때 sess.run(global_step)을 이용해야 합니다.
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
# 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트 파일에 저장합니다.
# global_step의 값은 저장할 파일의 이름에 추가로 붙게 되며, 텐서 변수 또는 숫자값을 넣어줄 수 있습니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

sess.close()

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-20
Step: 21,  Cost: 0.690
Step: 22,  Cost: 0.684
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
